# EE 379K: Lab 8

## Rohan Nagar and Wenyang Fu

In [1]:
import numpy as np

# Question 1

Some of the features we gave you are pure noise. Which ones? Find them and remove them.

# Question 2

Some of the features are noisy versions of other features. Which ones? Find them and remove them.

# Question 3

Retrain your models using this information. Merge models with your lab partner and report your best private score. Does removing the noise features improve your performance?

# Question 4

Write a post on the inclass Kaggle forums that contains some cool data science. Include your post in your report. Your post can be written by two people like your lab report.

# Question 5

Have a look here: 

https://www.kaggle.com/c/GiveMeSomeCredit

What is the best private score you can get for this competition? Include a screenshot of your submissions with scores in your lab report.